# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

In [1]:
import pandas as pd

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [7]:
from nltk.tokenize import word_tokenize

df = pd.read_csv('preprocessed_descriptions.csv', header=0)
all_words = []
for text in df['preprocessed_descriptions']:
    if isinstance(text, str):
        words = word_tokenize(text)
        all_words.extend(words)

unique = set(all_words)

print(unique)

{'309199', 'ape', 'parboiling', 'ladybug', 'preapare', 'heaping', 'pitta', 'midwesterner', 'horsing', 'ingridnl', 'message', '262189', 'parmesean', 'concoct', 'smokehouse', 'beyda', 'nostrom', 'mulberry', 'seventh', 'schezwan', 'sorely', 'stockings', 'midweek', 'piroshky', 'verson', 'tyrol', 'mono', 'faucet', 'horde', 'baaa', 'jollof', 'apologies', 'josephinas', 'oyster', 'rc', 'chiffon', 'chutzpah', 'photos', 'yield', 'slimlines', 'soft', 'chemotherapy', 'tal', 'tara', 'viable', 'history', 'jaxon', 'treat', 'stevenson', 'greenliving', 'submit', 'perfecto', 'refreshed', 'thestraight', 'robinson', 'written', 'maybe', 'bait', 'primary', 'becomes', 'allstar', 'courses', 'lune', 'provincial', 'ladling', 'superior', 'smilar', 'gertrude', 'tendency', 'licensed', 'planking', 'admittedly', 'farfelle', 'centennial', 'savings', 'sevilla', 'enticing', 'hope', 'rumor', 'cris', 'congo', 'shelton', 'miserably', 'celi', 'puma', 'rarebit', 'bothered', 'seperates', 'bagged', 'virtual', 'collecting', 's

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [12]:
import random
import editdistance

random_pairs = [(random.choice(list(unique)), random.choice(list(unique))) for _ in range(5)]

for pair in random_pairs:
    word1, word2 = pair
    distance = editdistance.eval(word1, word2)
    print(f"Расстояние редактирования между '{word1}' и '{word2}': {distance}")

Расстояние редактирования между 'layers' и 'giudia': 6
Расстояние редактирования между 'h' и 'thebackhomebakery': 16
Расстояние редактирования между 'aragon' и 'manjarblanco': 9
Расстояние редактирования между 'spolights' и 'hawk': 9
Расстояние редактирования между 'guy' и 'alcholic': 8


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [15]:
def find_closest_words(word, words, k):
    # Расстояния до каждого слова
    distances = [(w, editdistance.eval(word, w)) for w in words]
    # Сортировка
    sorted_distances = sorted(distances, key=lambda x: x[1])
    # Получение только слова, без числа близости
    closest_words = [w for w, _ in sorted_distances[:k]]
    return closest_words

input_word = "layers"
words_list = list(unique)
k_closest = 5

closest_words = find_closest_words(input_word, words_list, k_closest)
print(f"Слово: '{input_word}', ближайшие {k_closest}: {closest_words}")

Слово: 'layers', ближайшие 5: ['layers', 'players', 'layer', 'lagers', 'leaders']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [17]:
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

df = pd.DataFrame(columns=['word', 'stemmed_word', 'normalized_word'])
snowball_stemmer = SnowballStemmer('english')
wordnet_lemmatizer = WordNetLemmatizer()

for word in unique:
    stemmed_word = snowball_stemmer.stem(word)
    normalized_word = wordnet_lemmatizer.lemmatize(word)
    df.loc[word] = [word, stemmed_word, normalized_word]

print(df.head())

                  word stemmed_word normalized_word
309199          309199       309199          309199
ape                ape          ape             ape
parboiling  parboiling      parboil      parboiling
ladybug        ladybug      ladybug         ladybug
preapare      preapare      preapar        preapare


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).